In [1]:
! sudo apt-get update
! sudo mkdir -p /usr/share/man/man1
! sudo apt-get install -y openjdk-11-jdk
! pip install pyspark

Hit:1 http://deb.debian.org/debian buster InRelease
Hit:2 http://deb.debian.org/debian-security buster/updates InRelease
Hit:3 http://deb.debian.org/debian buster-updates InRelease




openjdk-11-jdk is already the newest version (11.0.18+10-1~deb10u1).
0 upgraded, 0 newly installed, 0 to remove and 12 not upgraded.
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [2]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName("Particionado").master(
    "local[7]"
).getOrCreate()

23/04/29 04:04:48 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [5]:
# crea por defecto una partición de 5
df = spark.range(0, 20)
df.rdd.getNumPartitions() 

5

In [6]:
# crear un rdd con una partición establecida
rdd1 = spark.sparkContext.parallelize((0, 20), 10)
rdd1.getNumPartitions()

10

In [10]:
# crear un rdd desde archivo y partisionarlo en 10
rdd_from_file = spark.sparkContext.textFile('/data/deporte.csv', 10)


In [11]:
rdd_from_file.getNumPartitions()

10

In [12]:
# guardar las particiones a una carpeta nueva
rdd_from_file.saveAsTextFile('/data/output_partitions')

In [19]:
!head -n 5 /data/output_partitions/part-00000

deporte_id,deporte
1,Basketball
2,Judo
3,Football
4,Tug-Of-War


In [20]:
rdd = spark.sparkContext.wholeTextFiles('/data/output_partitions/*')

In [21]:
rdd.take(2)

[('file:/work/output_partitions/part-00000',
  'deporte_id,deporte\n1,Basketball\n2,Judo\n3,Football\n4,Tug-Of-War\n5,Speed Skating\n6,Cross Country Skiing\n'),
 ('file:/work/output_partitions/part-00001',
  '7,Athletics\n8,Ice Hockey\n9,Swimming\n10,Badminton\n11,Sailing\n12,Biathlon\n13,Gymnastics\n14,Art Competitions\n')]

In [22]:
# guardar todos los archivos en una lista
list_files = rdd.mapValues(lambda f: f.split()).collect()

In [23]:
list_files

[('file:/work/output_partitions/part-00000',
  ['deporte_id,deporte',
   '1,Basketball',
   '2,Judo',
   '3,Football',
   '4,Tug-Of-War',
   '5,Speed',
   'Skating',
   '6,Cross',
   'Country',
   'Skiing']),
 ('file:/work/output_partitions/part-00001',
  ['7,Athletics',
   '8,Ice',
   'Hockey',
   '9,Swimming',
   '10,Badminton',
   '11,Sailing',
   '12,Biathlon',
   '13,Gymnastics',
   '14,Art',
   'Competitions']),
 ('file:/work/output_partitions/part-00002',
  ['15,Alpine',
   'Skiing',
   '16,Handball',
   '17,Weightlifting',
   '18,Wrestling',
   '19,Luge',
   '20,Water',
   'Polo']),
 ('file:/work/output_partitions/part-00003',
  ['21,Hockey',
   '22,Rowing',
   '23,Bobsleigh',
   '24,Fencing',
   '25,Equestrianism',
   '26,Shooting',
   '27,Boxing',
   '28,Taekwondo']),
 ('file:/work/output_partitions/part-00004',
  ['29,Cycling',
   '30,Diving',
   '31,Canoeing',
   '32,Tennis',
   '33,Modern',
   'Pentathlon',
   '34,Figure',
   'Skating',
   '35,Golf']),
 ('file:/work/output

In [27]:
list_files = [l[0] for l in list_files]
list_files.sort()
list_files

['file:/work/output_partitions/part-00000',
 'file:/work/output_partitions/part-00001',
 'file:/work/output_partitions/part-00002',
 'file:/work/output_partitions/part-00003',
 'file:/work/output_partitions/part-00004',
 'file:/work/output_partitions/part-00005',
 'file:/work/output_partitions/part-00006',
 'file:/work/output_partitions/part-00007',
 'file:/work/output_partitions/part-00008',
 'file:/work/output_partitions/part-00009']

In [29]:
# crear un rdd a partir de unos files particinado
rdd2 = spark.sparkContext.textFile(','.join(list_files), 10).map(
    lambda l: l.split(",")
)

In [30]:
rdd2.take(2)

[['deporte_id', 'deporte'], ['1', 'Basketball']]